# <p style="background-color: #f5df18; padding: 10px;"> Foundations of Astronomical Data Science | **Join** </p>



### <strong>Instructor: <span style="color: darkblue;">Devontae C. Baxter</span></strong>

Estimated completion time: 🕚 90 minutes



<div style="display: flex;">
    <div style="flex: 1; margin-right: 100px;">
        <h2>Questions</h2>
        <ul>
            <li>How do we use JOIN to combine information from multiple tables?</li>
            <li>How can we make a selection within a joined table?</li>
            <li>How should we save the result?</li>
        </ul>
    </div>
    <div style="flex: 1;">
        <h2>Learning Objectives</h2>
        <ul>
            <li>Write ADQL queries involving JOIN operations.</li>
            <li>Save data in CSV format</li>
        </ul>
    </div>
</div>

This imports previously imported functions:

In [1]:
from astroquery.gaia import Gaia
import pandas as pd

from episode_functions import *

The following code loads in the data (instructions for downloading data can be
found in the [setup instructions](../learners/setup.md)).  You may need to add a the path
to the filename variable below (e.g. `filename = 'student_download/backup-data/gd1_data.hdf'`)


In [2]:
filename = 'gd1_data.hdf'
point_series = pd.read_hdf(filename, 'point_series')
sky_point_list = point_series['sky_point_list']
pmra_min = point_series['pmra_min']
pmra_max = point_series['pmra_max']
pmdec_min = point_series['pmdec_min']
pmdec_max = point_series['pmdec_max']
point_series

sky_point_list    135.306, 8.39862, 126.51, 13.4449, 163.017, 54...
pmra_min                                                       -6.7
pmra_max                                                         -3
pmdec_min                                                    -14.31
pmdec_max                                                     -11.2
dtype: object

The next step in our analysis is to
select candidate stars based on photometry data.
The following figure from the Price-Whelan and Bonaca paper is a color-magnitude diagram for
the stars selected based on proper motion:

<div>
<img src="https://datacarpentry.org/astronomy-python/fig/gd1-3.png" width="300"/>
</div>

In orange is a [stellar
isochrone](https://en.wikipedia.org/wiki/Stellar_isochrone), showing
where we expect the stars in GD-1 to fall based on the metallicity and
age of their original globular cluster.

By selecting stars in the shaded area, we can further distinguish the
main sequence of GD-1 from younger background stars.

# Outline
---

1. We will reload the candidate stars we identified in the previous
  episode.

2. Then we will run a query on the Gaia server that uploads the table
  of
  candidates and uses a `JOIN` operation to select photometry data for
  the candidate stars.

3. We will write the results to a file for use in the next episode.

# Getting photometry data
---

The Gaia dataset contains some photometry data, including the variable
`bp_rp`, which contains BP-RP color (the difference in mean flux
between the BP and RP bands).
We use this variable to select stars with `bp_rp` between -0.75 and 2,
which excludes many class M dwarf stars.

But we can do better than that. Assuming GD-1 is a globular cluster, all of the stars formed at the same
time from the same material, so the stars' photometric properties should be consistent with a single
isochrone in a color magnitude diagram.
We can use photometric color and apparent magnitude to select stars with the age and metal richness we expect in GD-1.
However, the broad Gaia photometric bands (G, BP, RP) are not optimized for this task, instead we will use the more narrow photometric bands available from the Pan-STARRS survey to obtain the `g-i` color and apparent `g`\-band magnitude.

Conveniently, the Gaia server provides data from Pan-STARRS as a table
in the same database we have been using, so we can access it by making
ADQL queries.

## 🔔 A caveat about matching stars between catalogs
---

In general, choosing a star from the Gaia catalog and finding the
corresponding star in the Pan-STARRS catalog is not easy.  This kind
of cross matching is not always possible, because a star might appear
in one catalog and not the other.  And even when both stars are
present, there might not be a clear one-to-one relationship between
stars in the two catalogs. Additional [catalog matching tools](https://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs) are available from the
Astropy coordinates package.

Fortunately, people have worked on this problem, and the Gaia
database includes cross-matching tables that suggest a best neighbor
in the Pan-STARRS catalog for many stars in the Gaia catalog.

[This document describes the cross matching process](https://gea.esac.esa.int/archive/documentation/GDR2/Catalogue_consolidation/chap_cu9val_cu9val/ssec_cu9xma/sssec_cu9xma_extcat.html).
Briefly, it uses a cone search to find possible matches in
approximately the right position, then uses attributes like color and
magnitude to choose pairs of observations most likely to be the same
star.


## The best neighbor table

So the hard part of cross-matching has been done for us.  Using the
results is a little tricky, but it gives us a chance to learn about
one of the most important tools for working with databases: "joining"
tables.

A "join" is an operation where you match up records from
one table with records from another table using as a "key" a piece of
information that is common to both tables, usually some kind of ID
code.

In this example:

- Stars in the Gaia dataset are identified by `source_id`.

- Stars in the Pan-STARRS dataset are identified by `obj_id`.

For each candidate star we have selected so far, we have the
`source_id`; the goal is to find the `obj_id` for the same star
in the Pan-STARRS catalog.

To do that we will:

1. Use the `JOIN` operator to look up each Pan-STARRS `obj_id` for the stars
  we are interested in in the` panstarrs1_best_neighbour` table using the `source_id`s
  that we have already identified.

2. Use the `JOIN` operator again to look up the Pan-STARRS photometry for these stars
  in the `panstarrs1_original_valid` table using the` obj_ids` we just identified.

Before we get to the `JOIN` operation, we will explore these tables.

## 🔔 British vs American Spelling of Neighbour

The Gaia database was created and is maintained by the European Space Astronomy Center.
For this reason, the table spellings use the British spelling of neighbour
(with a "u"). Do not forget to include it in your table names in the queries below.


Here is the metadata for `panstarrs1_best_neighbour`.

And here are the columns.

Here is the [documentation for these
variables](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_crossmatches/ssec_dm_panstarrs1_best_neighbour.html).

The ones we will use are:

- `source_id`, which we will match up with `source_id` in the Gaia table.

- `best_neighbour_multiplicity`, which indicates how many sources in
  Pan-STARRS are matched with the same probability to this source in Gaia.

- `number_of_mates`, which indicates the number of *other* sources in
  Gaia that are matched with the same source in Pan-STARRS.

- `original_ext_source_id`, which we will match up with `obj_id` in
  the Pan-STARRS table.

Ideally, `best_neighbour_multiplicity` should be 1 and `number_of_mates`
should be 0; in that case, there is a one-to-one match between the
source in Gaia and the corresponding source in Pan-STARRS.

## 🔔 Number of neighbors

The table also contains `number_of_neighbours` which is the
number of stars in Pan-STARRS that match in terms of position, before
using other criteria to choose the most likely match.  But we are more
interested in the final match, using both criteria.

Here is a query that selects these columns and returns the first 5 rows.

## The Pan-STARRS table

Now that we know the Pan-STARRS `obj_id`, we are ready to match this to the photometry in the
`panstarrs1_original_valid` table. Here is the metadata for the table that contains the Pan-STARRS data.

And here are the columns.

Here is the [documentation for these variables](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_external_catalogues/ssec_dm_panstarrs1_original_valid.html) .

The ones we will use are:

- `obj_id`, which we will match up with `original_ext_source_id` in
  the best neighbor table.

- `g_mean_psf_mag`, which contains mean magnitude from the `g` filter.

- `i_mean_psf_mag`, which contains mean magnitude from the `i` filter.

Here is a query that selects these variables and returns the first 5 rows.

## Joining tables
---

The following figure shows how these tables are related.

- The orange circles and arrows represent the first `JOIN` operation,
  which takes each `source_id` in the Gaia table and finds the same
  value of `source_id` in the best neighbor table.

- The blue circles and arrows represent the second `JOIN` operation,
  which takes each `original_ext_source_id` in the best neighbor table and finds
  the same value of `obj_id` in the PanSTARRS photometry table.

There is no guarantee that the corresponding rows of these tables are
in the same order, so the `JOIN` operation involves some searching.
However, ADQL/SQL databases are implemented in a way that makes this
kind of search efficient.
If you are curious, you can [read more about
it](https://chartio.com/learn/databases/how-does-indexing-work/).

<div>
<img src="https://datacarpentry.org/astronomy-python/fig/join.png" width="1000"/>
</div>


Now we will get to the details of performing a `JOIN` operation.

We are about to build a complex query using software that doesn't provide us with any helpful information for debugging.
For this reason we are going to start with a simplified version of what we want to do until we are sure  we are joining
the tables correctly, then we will slowly add more layers of complexity, checking at each stage that our query still works.
As a starting place, we will go all the way back to the cone search from lesson 2.

And we will run it, to make sure we have a working query to build on.


Now we can start adding features.
First, we will replace `source_id` with the format specifier `columns` so that we can alter what columns we
want to return without having to modify our base query:

As a reminder, here are the columns we want from the Gaia table:

```python
columns = 'source_id, ra, dec, pmra, pmdec'
```


We run the query again.

## Adding the best neighbor table

Now we are ready for the first join.
The join operation requires two clauses:

- `JOIN` specifies the name of the table we want to join with, and

- `ON` specifies how we will match up rows between the tables.

In this example, we join with `gaiadr2.panstarrs1_best_neighbour AS best`, which means we can refer to the best neighbor table with the
abbreviated name `best`, which will save us a lot of typing. Similarly,
we will be referring to the `gaiadr2.gaia_source` table by the abbreviated name `gaia`.

The `ON` clause indicates that we will match up the `source_id`
column from the Gaia table with the `source_id` column from the best
neighbor table.

```python
neighbours_base_query = """SELECT 
{columns}
FROM gaiadr2.gaia_source AS gaia
JOIN gaiadr2.panstarrs1_best_neighbour AS best
  ON gaia.source_id = best.source_id
WHERE 1=CONTAINS(
  POINT(gaia.ra, gaia.dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""
```

## 🔔 SQL detail

In this example, the `ON` column has the same name in both tables, so we could replace the `ON` clause
with a simpler [`USING`clause](https://docs.oracle.com/javadb/10.8.3.0/ref/rrefsqljusing.html):

```sql
USING(source_id)
```


Now that there is more than one table involved, we can't use simple
column names any more; we have to use **qualified column names**.
In other words, we have to specify which table each column is in.
The column names do not have to be the same and, in fact, in the next join they will not be.
That is one of the reasons that we explicitly specify them.
Here is the complete query, including the columns we want from the Gaia
and best neighbor tables. Here you can start to see that using the abbreviated names
is making our query easier to read and requires less typing for us. In addition to the
spatial coordinates and proper motion, we are going to return the `best_neighbour_multiplicity`
and `number_of_mates` columns from the `panstarrs1_best_neighbour` table in order to evaluate the quality
of the data that we are using by evaluating the number of one-to-one matches between the catalogs.
Recall that `best_neighbour_multiplicity` tells us the number of PanSTARRs
objects that match a Gaia object and `number_of_mates` tells us the number of Gaia objects that match a
PanSTARRs object.


This result has fewer rows than the previous result.
That is because there are sources in the Gaia table with no
corresponding source in the Pan-STARRS table.

By default, the result of the join only includes rows where the same
`source_id` appears in both tables.
This default is called an "inner" join because the results include
only the intersection of the two tables.
[You can read about the other kinds of join
here](https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/).

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Adding the Pan-STARRS table (15 Minutes) </p>
---

Now we are ready to bring in the Pan-STARRS table.  Starting with the
previous query, add a second `JOIN` clause that joins with
`gaiadr2.panstarrs1_original_valid`, gives it the abbreviated name
`ps`, and matches `original_ext_source_id` from the best neighbor
table with `obj_id` from the Pan-STARRS table.

Add `g_mean_psf_mag` and `i_mean_psf_mag` to the column list, and run the query.
The result should contain 490 rows and 9 columns.

In [ ]:
### your answer here ###

## Selecting by coordinates and proper motion

We are now going to replace the cone search with the GD-1 selection that we built in previous episodes.
We will start by making sure that our previous query works, then add in the `JOIN`.
Now we will bring in the `WHERE` clause from the previous episode, which
selects sources based on parallax, BP-RP color, sky coordinates, and
proper motion.

Here is `candidate_coord_pm_query_base` from the previous episode.

```python
candidate_coord_pm_query_base = """SELECT 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({sky_point_list}))
  AND pmra BETWEEN {pmra_min} AND  {pmra_max}
  AND pmdec BETWEEN {pmdec_min} AND {pmdec_max}
"""
```

Now we can assemble the query using the sky point list and proper motion range we compiled in episode 5.

We run it to make sure we are starting with a working query.

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise (15 Minutes) </p>
---

Create a new query base called `candidate_join_query_base` that combines the `WHERE`
clauses from the previous query with the `JOIN` clauses for the best
neighbor and Pan-STARRS tables.
Format the query base using the column names in `column_list`, and
call the result `candidate_join_query`.

Hint: Make sure you use qualified column names everywhere!

Run your query and download the results.  The table you get should
have 4300 rows and 9 columns.

In [ ]:
### your answer here ####

# Checking the match
---

To get more information about the matching process, we can inspect
`best_neighbour_multiplicity`, which indicates for each star in Gaia
how many stars in Pan-STARRS are equally likely matches.

Most of the values are `1`, which is good; that means
that for each candidate star we have identified exactly one source in
Pan-STARRS that is likely to be the same star.

To check whether there are any values other than `1`, we can convert
this column to a Pandas `Series` and use `describe`, which we saw in
in episode 3.

```python
multiplicity = pd.Series(candidate_table['best_neighbour_multiplicity'])
multiplicity.describe()
```

```output
count    4300.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64
```

In fact, `1` is the only value in the `Series`, so every candidate
star has a single best match.

## 🔔 Numpy Mask Warning

You may see a warning that ends with the following phrase:

`site-packages/numpy/lib/function_base.py:4650:`

`UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.`
`arr.partition(`

This is because astroquery is returning a table with masked columns (which are really fancy masked numpy arrays).
When we turn this column into a pandas Series, it maintains its mask. Describe calls numpy functions to perform statistics.
Numpy recently implemented this warning to let you know that the mask is not being considered in the calculation
its performing.


Similarly, `number_of_mates` indicates the number of *other* stars in
Gaia that match with the same star in Pan-STARRS.

```python
mates = pd.Series(candidate_table['number_of_mates'])
mates.describe()
```

```output
count    4300.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64
```

All values in this column are `0`, which means that for each match we
found in Pan-STARRS, there are no other stars in Gaia that also match.

# Saving the DataFrame
---

We can make a `DataFrame` from our Astropy `Table` and save our results so we can pick up where we left off
without running this query again. Once again, we will make use of our `make_dataframe` function.

```python
candidate_df = make_dataframe(candidate_table)
```

The HDF5 file should already exist, so we'll add `candidate_df` to it.

```python
filename = 'gd1_data.hdf'

candidate_df.to_hdf(filename, 'candidate_df')
```

We can use `getsize` to confirm that the file exists and check the size:

```python
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB
```

```output
15.422508239746094
```

# Another file format - CSV
--- 

Pandas can write a variety of other formats, [which you can read about
here](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).
We won't cover all of them, but one other important one is
[CSV](https://en.wikipedia.org/wiki/Comma-separated_values), which
stands for "comma-separated values".

CSV is a plain-text format that can be read and written by pretty much
any tool that works with data.  In that sense, it is the "least common
denominator" of data formats.

However, it has an important limitation: some information about the
data gets lost in translation, notably the data types.  If you read a
CSV file from someone else, you might need some additional information
to make sure you are getting it right.

Also, CSV files tend to be big, and slow to read and write.

With those caveats, here is how to write one:

the kind of thing that can go wrong with CSV files.

We can check the file size like this:

We can read the CSV file back like this:

We will compare the first few rows of `candidate_df` and `read_back_csv`

# Summary

---

In this episode, we used database `JOIN` operations to select
photometry data for the stars we've identified as candidates to be in
GD-1.

In the next episode, we will use this data for a second round of
selection, identifying stars that have photometry data consistent with
GD-1.

# <p style="background-color: #f5df18; padding: 10px;"> 🗝️ Key points</p>

---

- Use `JOIN` operations to combine data from multiple tables in a database, using some kind of identifier to match up records from one table with records from another. This is another example of a practice we saw in the previous notebook, moving the computation to the data.
- For most applications, saving data in FITS or HDF5 is better than CSV.  FITS and HDF5 are binary formats, so the files are usually smaller, and they store metadata, so you don't lose anything when you read the file back.
- On the other hand, CSV is a 'least common denominator' format; that is, it can be read by practically any application that works with data.